<h1 style="line-height:50px;;background:#9eb5db;border-radius:15px;height:50px;text-align:center;border:2px ridge;
color:white;">Data Prepration</h1>

<div style="background:#9eb5db;border-radius:15px;"><h2 style="line-height:50px;height:50px;margin-left:20px;color:white">Requirements
</h2></div>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


<div style="background:#9eb5db;border-radius:15px;"><h2 style="line-height:50px;height:150px;margin-left:20px;color:white">Problem: Pre-processing on all columns to reach a usable format for machine learning algorithms. In this exercise, the target column is not considered and it is only an exercise for pre-processing on the dataset.
</h2></div>

In [2]:
df = pd.read_csv("flights.csv")

In [3]:
df.head()

,id,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,...,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour,name
0,0,2013,1,1,517.0,515,2.0,830.0,819,11.0,...,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01 05:00:00,United Air Lines Inc.
1,1,2013,1,1,533.0,529,4.0,850.0,830,20.0,...,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01 05:00:00,United Air Lines Inc.
2,2,2013,1,1,542.0,540,2.0,923.0,850,33.0,...,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01 05:00:00,American Airlines Inc.
3,3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,...,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01 05:00:00,JetBlue Airways
4,4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,...,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01 06:00:00,Delta Air Lines Inc.


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336776 entries, 0 to 336775
Data columns (total 21 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              336776 non-null  int64  
 1   year            336776 non-null  int64  
 2   month           336776 non-null  int64  
 3   day             336776 non-null  int64  
 4   dep_time        328521 non-null  float64
 5   sched_dep_time  336776 non-null  int64  
 6   dep_delay       328521 non-null  float64
 7   arr_time        328063 non-null  float64
 8   sched_arr_time  336776 non-null  int64  
 9   arr_delay       327346 non-null  float64
 10  carrier         336776 non-null  object 
 11  flight          336776 non-null  int64  
 12  tailnum         334264 non-null  object 
 13  origin          336776 non-null  object 
 14  dest            336776 non-null  object 
 15  air_time        327346 non-null  float64
 16  distance        336776 non-null  int64  
 17  hour      

In [5]:
# Delete columns that have no data 
df.columns

Index(['id', 'year', 'month', 'day', 'dep_time', 'sched_dep_time', 'dep_delay',
       'arr_time', 'sched_arr_time', 'arr_delay', 'carrier', 'flight',
       'tailnum', 'origin', 'dest', 'air_time', 'distance', 'hour', 'minute',
       'time_hour', 'name'],
      dtype='object')

In [6]:
# id, carrier, tailnum is not significant data brcause they're just have numerci label or label.
# time_hour consists of columns dep_time and month and day but time_hour in utc format and the other in local.

trim_df = df.drop(['id', 'year', 'carrier', 'tailnum', 'time_hour'], axis=1)

In [7]:
trim_df

,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,flight,origin,dest,air_time,distance,hour,minute,name
0,1,1,517.0,515,2.0,830.0,819,11.0,1545,EWR,IAH,227.0,1400,5,15,United Air Lines Inc.
1,1,1,533.0,529,4.0,850.0,830,20.0,1714,LGA,IAH,227.0,1416,5,29,United Air Lines Inc.
2,1,1,542.0,540,2.0,923.0,850,33.0,1141,JFK,MIA,160.0,1089,5,40,American Airlines Inc.
3,1,1,544.0,545,-1.0,1004.0,1022,-18.0,725,JFK,BQN,183.0,1576,5,45,JetBlue Airways
4,1,1,554.0,600,-6.0,812.0,837,-25.0,461,LGA,ATL,116.0,762,6,0,Delta Air Lines Inc.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336771,9,30,NaN,1455,NaN,NaN,1634,NaN,3393,JFK,DCA,NaN,213,14,55,Endeavor Air Inc.
336772,9,30,NaN,2200,NaN,NaN,2312,NaN,3525,LGA,SYR,NaN,198,22,0,Endeavor Air Inc.
336773,9,30,NaN,1210,NaN,NaN,1330,NaN,3461,LGA,BNA,NaN,764,12,10,Envoy Air
336774,9,30,NaN,1159,NaN,NaN,1344,NaN,3572,LGA,CLE,NaN,419,11,59,Envoy Air


In [8]:
trim_df.isnull().sum()

month                0
day                  0
dep_time          8255
sched_dep_time       0
dep_delay         8255
arr_time          8713
sched_arr_time       0
arr_delay         9430
flight               0
origin               0
dest                 0
air_time          9430
distance             0
hour                 0
minute               0
name                 0
dtype: int64

In [9]:
trim_df.isna().sum()

month                0
day                  0
dep_time          8255
sched_dep_time       0
dep_delay         8255
arr_time          8713
sched_arr_time       0
arr_delay         9430
flight               0
origin               0
dest                 0
air_time          9430
distance             0
hour                 0
minute               0
name                 0
dtype: int64

In [10]:
# Seperate Numeric Columns and Non-Numerical Columns

X = trim_df.drop(['origin', 'dest', 'name'], axis = 1)
Y = trim_df.drop(X.columns, axis=1)
print(X.columns)
print(Y.columns)

Index(['month', 'day', 'dep_time', 'sched_dep_time', 'dep_delay', 'arr_time',
       'sched_arr_time', 'arr_delay', 'flight', 'air_time', 'distance', 'hour',
       'minute'],
      dtype='object')
Index(['origin', 'dest', 'name'], dtype='object')


In [11]:
# Replace Nearest Number For Nan or Null in Numeric Data

si = SimpleImputer(missing_values = np.nan, strategy = 'mean')
si.fit(X)
X_encoded = si.transform(X)

In [12]:
# OneHotEncoding on non-numeric Cols

ohe = OneHotEncoder()
Y_encoded = ohe.fit_transform(Y)
Y_encoded = Y_encoded.toarray()
print(Y_encoded)

[[1. 0. 0. ... 0. 1. 0.]
 [0. 0. 1. ... 0. 1. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]


In [13]:
# StandardScalor on Numerci Data

ss = StandardScaler()
X_encoded = ss.fit_transform(X_encoded)

In [14]:
## this cell run in scikit-learn == 1.2.2 

# # create a new DataFrame with the concatenated data
# new_df = np.concatenate([X_encoded, Y_encoded], axis=1)

# cols = list(X.columns) + list(ohe.get_feature_names(Y.columns))
# df_concatenated = pd.DataFrame(new_df, columns=cols)

# # print the concatenated DataFrame
# print(df_concatenated.head())